In [1]:
import pandas as pd
import pennylane as qml
import numpy as np

### Lectura del fichero CSV

In [2]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

#Filtrado para quitar los datos string
filtered_data_only_int = data.iloc[:, 2:-2]

#Construir dataframe binario
df_binario = filtered_data_only_int.applymap(lambda x: 1 if x > 0 else 0)

#Data frame con primeras 20 columnas
df_binario_0_20 = df_binario.iloc[:, 0:20]

#Separar por vectores train y test
train_data = data[data['Set'] == 'Train']
test_data = data[data['Set'] == 'Test']

In [4]:
N = len(df_binario_0_20.columns[:20])
wires = range(N)
dev = qml.device("default.qubit", wires)

In [ ]:
@qml.qnode(dev)
def circuit(vector):
    qml.BasisEmbedding(features=vector, wires=range(N))
    return qml.state()

In [7]:
##Primera linea como lista
X = df_binario_0_20.iloc[0].to_list()
X_1 = df_binario_0_20.iloc[1].to_list()

In [15]:
print("Estado_0 ")
estado_0 = circuit(X)
print(qml.draw(circuit, expansion_strategy="device")(X))
print("\nEstado_1")
estado_1 = circuit(X_1)
print(qml.draw(circuit, expansion_strategy="device")(X_1))

Estado_0 
 4: ──X─┤  State
 6: ──X─┤  State
 7: ──X─┤  State
 8: ──X─┤  State
14: ──X─┤  State
15: ──X─┤  State

Estado_1
 6: ──X─┤  State
 7: ──X─┤  State
 8: ──X─┤  State
15: ──X─┤  State
